In [19]:
# import stanza
# # stanza.download('en')

# nlp_stanza = stanza.Pipeline('en',processors= 'tokenize,ner')
# nlp_stanza.processors['ner'].get_known_tags()

In [20]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [21]:
list_ners = list(nlp.get_pipe('ner').labels)

## Comparison

In [22]:
import numpy as np
import pandas as pd

Loading the dfs

In [23]:
df_entities_spacy = pd.read_csv(r"df_entities_spacy_processed.csv")

In [24]:
df_entities_stanza = pd.read_csv(r"df_entities_stanza_processed.csv")

In [25]:
list_people = np.unique(df_entities_spacy["subject"])

In [26]:
df_entities_spacy.columns

Index(['text', 'n_word', 'start_pos', 'end_pos', 'label', 'biography',
       'subject', 'source_index', 'category', 'range_span'],
      dtype='object')

### FX

This works, just need to complete it a lil bit

In [27]:
filter_df_spacy = df_entities_spacy[df_entities_spacy["subject"] == list_people[0]][["subject", "text", "label", "start_pos", "range_span"]]


In [28]:
filter_df_stanza = df_entities_stanza[df_entities_stanza["subject"] == list_people[0]][["subject", "text", "label", "start_pos", "range_span"]]


In [29]:
# filter_df_stanza

In [30]:
# filter_df_spacy

In [31]:
results = []

for person in list_people:
    
    filter_df_spacy = df_entities_spacy[df_entities_spacy["subject"] == person][["subject", "text", "label", "start_pos", "range_span"]]
    filter_df_stanza = df_entities_stanza[df_entities_stanza["subject"] == person][["subject", "text", "label", "start_pos", "range_span"]]
    
    i = 0
    j = 0

    partial_agreement = 0
    total_agreement = 0
    label_agreement = 0
    
    # doing this with list comprehendion made it so that all the nested dictionaries would update when one updated
    dict_ners = {}
    for ner in list_ners:
        dict_ners[ner] = {"Total": 0, "Agreed": 0}
    
    # print(dict_ners)
    # print(dict_ners.keys())

    while i+1 < len(filter_df_spacy) and j+1 < len(filter_df_stanza): # adjusting for mismatch between indexing and length

        # print(i, j)
        token_text_spacy = filter_df_spacy.iloc[i]["text"]
        token_text_stanza = filter_df_stanza.iloc[j]["text"]
        
        # print(token_text_spacy, token_text_stanza)
        # print(len(token_text_spacy), len(token_text_stanza))
        
        token_span_spacy = filter_df_spacy.iloc[i]["range_span"].replace("[", "").replace("]", "").replace("\n", "").replace("  ", " ").strip().split(" ")
        token_span_stanza = filter_df_stanza.iloc[j]["range_span"].replace("[", "").replace("]", "").replace("\n", "").replace("  ", " ").strip().split(" ")

        # print(token_span_spacy, token_span_stanza)
        # print(len(token_span_spacy), len(token_span_stanza))

        pos_token_spacy = filter_df_spacy.iloc[i]["label"]
        pos_token_stanza = filter_df_stanza.iloc[j]["label"]

        list_agreement_span = [index for index in token_span_spacy if index in token_span_spacy and index in token_span_stanza]
        
        if token_span_spacy == token_span_stanza:
            # print(token_text_spacy, token_text_stanza)
            total_agreement = total_agreement + 1
            
            # print(pos_token_spacy)
            # print(pos_token_stanza)
            # print(dict_ners[pos_token_spacy])
            # print(dict_ners[pos_token_stanza])
            # print(dict_ners)
            
            dict_ners[pos_token_spacy]["Total"] = dict_ners[pos_token_spacy]["Total"] + 1
            dict_ners[pos_token_stanza]["Total"] = dict_ners[pos_token_stanza]["Total"] + 1
            
            if pos_token_spacy == pos_token_stanza:
                label_agreement = label_agreement + 1
                
                dict_ners[pos_token_spacy]["Agreed"] = dict_ners[pos_token_spacy]["Agreed"] + 1
                dict_ners[pos_token_stanza]["Agreed"] = dict_ners[pos_token_stanza]["Agreed"] + 1
                
            i = i + 1
            j = j + 1
            
            
        elif len(list_agreement_span) > 1:
            # print(token_text_spacy, token_text_stanza)
            partial_agreement = partial_agreement + 1 
            
            dict_ners[pos_token_spacy]["Total"] = dict_ners[pos_token_spacy]["Total"] + 1
            dict_ners[pos_token_stanza]["Total"] = dict_ners[pos_token_stanza]["Total"] + 1
            
            if pos_token_spacy == pos_token_stanza:
                label_agreement = label_agreement + 1
                
                dict_ners[pos_token_spacy]["Agreed"] = dict_ners[pos_token_spacy]["Agreed"] + 1
                dict_ners[pos_token_stanza]["Agreed"] = dict_ners[pos_token_stanza]["Agreed"] + 1
                
            i = i + 1
            j = j + 1
            
        else:
            if token_span_spacy[-1] < token_span_stanza[-1]:
                i = i + 1
                j = j
            elif token_span_spacy[-1] > token_span_stanza[-1]:
                i = i
                j = j + 1
                
        # print(dict_ners)
        # print(dict_ners.keys())
        # print("DONE WITH TOKENS", token_text_spacy, token_text_stanza)
        
    total = len(filter_df_spacy) + len(filter_df_stanza) - partial_agreement - total_agreement
    per_agreement = round(((total_agreement+partial_agreement)/total), 2)
    
    results_person = {"subject":person,
                      "total_entities": total,
                      "percent_partial_or_total_agreement_span":per_agreement,  # over the total amount of entities
                      "total_agreement_per": round((total_agreement/total), 2),  # over the total amount of entities
                      "partial_agreement_per": round((partial_agreement/total), 2),  # over the total amount of entities
                      "ner_breakdown": dict_ners}
    
    for key in results_person["ner_breakdown"].keys():
        results_person["total_ner_aggregate"] = round((sum([results_person["ner_breakdown"][key]["Agreed"] for key in results_person["ner_breakdown"].keys()])  / sum([results_person["ner_breakdown"][key]["Total"] for key in results_person["ner_breakdown"].keys()])), 2)
        if results_person["ner_breakdown"][key]["Total"] != 0: # avoiding zero division
            agreement_label_perc = round((results_person["ner_breakdown"][key]["Agreed"] / results_person["ner_breakdown"][key]["Total"]), 2)
            results_person["ner_breakdown"][key]["agreement_per"] = agreement_label_perc
            
        else:
            results_person["ner_breakdown"][key]["agreement_per"] = "Not applicable"
            
    results.append(results_person)
    
    # print(person)
    # print(" Same annotations:", total_agreement)
    # print(" Partial annotations:", partial_agreement)
    # print(" Total:", total)
    # print(" Perc. total or partial annotations:", per_agreement)
    # print()

In [32]:
results[0]

{'subject': 'Aage Bohr',
 'total_entities': 328,
 'percent_partial_or_total_agreement_span': 0.76,
 'total_agreement_per': 0.68,
 'partial_agreement_per': 0.08,
 'ner_breakdown': {'CARDINAL': {'Total': 26,
   'Agreed': 22,
   'agreement_per': 0.85},
  'DATE': {'Total': 125, 'Agreed': 122, 'agreement_per': 0.98},
  'EVENT': {'Total': 0, 'Agreed': 0, 'agreement_per': 'Not applicable'},
  'FAC': {'Total': 2, 'Agreed': 2, 'agreement_per': 1.0},
  'GPE': {'Total': 66, 'Agreed': 42, 'agreement_per': 0.64},
  'LANGUAGE': {'Total': 0, 'Agreed': 0, 'agreement_per': 'Not applicable'},
  'LAW': {'Total': 0, 'Agreed': 0, 'agreement_per': 'Not applicable'},
  'LOC': {'Total': 1, 'Agreed': 0, 'agreement_per': 0.0},
  'MONEY': {'Total': 2, 'Agreed': 2, 'agreement_per': 1.0},
  'NORP': {'Total': 26, 'Agreed': 26, 'agreement_per': 1.0},
  'ORDINAL': {'Total': 10, 'Agreed': 10, 'agreement_per': 1.0},
  'ORG': {'Total': 84, 'Agreed': 62, 'agreement_per': 0.74},
  'PERCENT': {'Total': 0, 'Agreed': 0, 'agr

In [33]:
keys_aggregate = ["percent_partial_or_total_agreement_span", "total_agreement_per", "partial_agreement_per", "total_ner_aggregate"]
results_aggregate_clean = []
for result in results:
    dict_result = {k:v for k,v in result.items() if k in keys_aggregate}
    for k, val in result["ner_breakdown"].items():
        dict_result[k] = result["ner_breakdown"][k]["agreement_per"]
    results_aggregate_clean.append(dict_result)

In [34]:
# results_aggregate_clean

In [35]:
df_results = pd.DataFrame(results_aggregate_clean)

In [36]:
df_results.to_csv("results_ner_comparison.csv", index=False)